# Notebook 1 – Agenten

Dieses Notebook veranschaulicht das Agentenmodell. Es ist als Live-Demo gedacht und nutzt entweder **aima-python** oder eine minimale Eigenimplementierung.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)

## Setup aima-python (optional)

Wenn du aima-python lokal oder auf Kaggle nutzen willst, kannst du das Repo installieren. Falls Installation nicht möglich ist, überspringe diesen Block und nutze die Minimal-Demo darunter.

In [18]:
!git clone --depth 1 https://github.com/aimacode/aima-python.git
%cd aima-python

Cloning into 'aima-python'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 214 (delta 21), reused 114 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (214/214), 7.37 MiB | 18.91 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/kaggle/working/aima-python/aima-python/aima-python


In [19]:
import sys, os
sys.path.append(os.getcwd())  # damit aima-python Module importierbar sind

## Agent und Umgebung als Minimalmodell

Ein einfacher Reflex-Agent reagiert nur auf die aktuelle Wahrnehmung.

### Pseudocode: Minimaler Agent in einer 1D-Umgebung

**Idee:** Ein Agent bewegt sich in einer eindimensionalen Welt mit Feldern `0 ... size-1`.  
Ziel ist ein bestimmtes Feld `goal`. Pro Schritt gibt es eine kleine Strafe, beim Ziel eine Belohnung.

**Umgebung (Environment)**
1. Setze Startzustand: `pos = 0`
2. Wiederhole für jeden Schritt:
   - Nimm eine Aktion `action` entgegen (links oder rechts)
   - Aktualisiere Position: `pos = pos + action`
   - Begrenze Position auf gültige Felder: `pos ∈ [0, size-1]`
   - Wenn `pos == goal`:
     - `reward = 1`
     - `done = True`
   - Sonst:
     - `reward = step_penalty` (z. B. `-0.01`)
     - `done = False`
   - Gib zurück: `(pos, reward, done)`

**Agent (Reflex-Agent)**
- Nutzt nur die aktuelle Beobachtung `obs`
- Keine Erinnerung, kein Lernen
- Regel: „Gehe immer nach rechts“

**Episode (Simulationslauf)**
1. `obs = env.reset()`
2. `total = 0`
3. Für maximal `T` Schritte:
   - `action = agent(obs)`
   - `(obs, reward, done) = env.step(action)`
   - `total += reward`
   - Wenn `done`: stoppe
4. Ergebnis: `total` (Gesamtreward) und finale Position `obs`

In [37]:
from dataclasses import dataclass

@dataclass
class SimpleEnv:
    """
    Minimal-Umgebung (1D):
    - Zustände sind Positionen auf einer Linie: 0, 1, ..., size-1
    - Der Agent startet immer bei pos=0
    - Das Ziel ist ein bestimmtes Feld 'goal'
    """
    size: int = 5     # Anzahl der Felder in der Welt
    goal: int = 4     # Zielposition (muss zwischen 0 und size-1 liegen)

    def reset(self):
        """
        Startet eine neue Episode.
        Setzt die Position auf den Startzustand und liefert die erste Beobachtung zurück.
        """
        self.pos = 0
        return self.pos  # Beobachtung = aktuelle Position

    def step(self, action):
        """
        Führt einen Zeitschritt aus.
        action: -1 bedeutet "nach links", +1 bedeutet "nach rechts"

        Rückgabe:
        - neue Beobachtung (pos)
        - reward (Belohnung/Strafe)
        - done (True, wenn Episode beendet ist)
        """
        # 1) Positionsupdate durch Aktion
        # Beispiel: pos=2, action=+1 -> pos wird 3
        self.pos = self.pos + action

        # 2) Begrenzung auf gültige Felder (Randbedingungen):
        # - unter 0 fällt der Agent nicht (bleibt bei 0)
        # - über size-1 steigt er nicht (bleibt bei size-1)
        self.pos = max(0, min(self.size - 1, self.pos))

        # 3) Reward-Definition:
        # - Ziel erreicht: +1
        # - sonst: kleine Schrittstrafe (damit kurze Wege "besser" sind)
        reward = 1 if self.pos == self.goal else -0.01

        # 4) Abbruchbedingung:
        # Episode endet, sobald das Ziel erreicht ist
        done = (self.pos == self.goal)

        return self.pos, reward, done


def reflex_agent(obs):
    """
    Reflex-Agent:
    - nutzt nur die aktuelle Beobachtung (obs), hier: Position
    - hat kein Gedächtnis und lernt nicht
    - Regel: immer nach rechts gehen
    """
    return +1


# --- Simulation einer Episode ---
env = SimpleEnv(size=5, goal=4)  # Umgebung erzeugen
obs = env.reset()                # Episode starten, obs=Startposition
total = 0                        # Gesamtreward sammeln

for t in range(20):              # Sicherheitslimit: max. 20 Schritte
    action = reflex_agent(obs)   # Agent wählt Aktion basierend auf obs
    obs, r, done = env.step(action)  # Umgebung führt Aktion aus
    total += r                   # Reward aufaddieren

    if done:                     # Ziel erreicht -> Episode beenden
        break

total, obs  # Ausgabe: (Gesamtreward, Endposition)

(0.97, 4)

## Play-Zelle

In [38]:
try:
    import ipywidgets as w
    from IPython.display import display, clear_output
except Exception as e:
    print("ipywidgets nicht verfügbar:", e)
    raise

import random
from dataclasses import dataclass

@dataclass
class SimpleEnvW:
    size: int
    goal: int
    step_penalty: float

    def reset(self):
        self.pos = 0
        return self.pos

    def step(self, action):
        self.pos = max(0, min(self.size-1, self.pos + action))
        reward = 1.0 if self.pos == self.goal else self.step_penalty
        done = self.pos == self.goal
        return self.pos, reward, done

def run(p_random=0.2, env_size=7, step_penalty=-0.02, episodes=200):
    env = SimpleEnvW(size=env_size, goal=env_size-1, step_penalty=step_penalty)

    def agent(obs):
        if random.random() < p_random:
            return random.choice([-1, +1])
        return +1

    rets=[]
    for _ in range(episodes):
        s=env.reset()
        total=0.0
        for t in range(60):
            a=agent(s)
            s,r,done=env.step(a)
            total += r
            if done: break
        rets.append(total)

    clear_output(wait=True)
    display(pd.Series(rets).describe())
    plt.figure()
    plt.hist(rets, bins=20)
    plt.xlabel("Return")
    plt.ylabel("Häufigkeit")
    plt.show()

p = w.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.05, description="p_random")
n = w.IntSlider(value=7, min=5, max=15, step=1, description="env_size")
sp = w.FloatSlider(value=-0.02, min=-0.2, max=0.0, step=0.01, description="penalty")
ep = w.IntSlider(value=200, min=50, max=800, step=50, description="episodes")

ui = w.VBox([p, n, sp, ep])
out = w.interactive_output(run, {"p_random": p, "env_size": n, "step_penalty": sp, "episodes": ep})
display(ui, out)


Output()

### Interpretation der Skala und Parameterwirkung

- **Return pro Episode** ist die Summe aller Rewards innerhalb einer Episode. Die Skala ergibt sich aus **Zielreward (typisch 1.0)** plus den **Schrittstrafen** bis zum Erreichen des Ziels.
- **step_penalty (negativ)** verschiebt die Verteilung nach unten: Je stärker die Schrittstrafe, desto kleiner der Return, besonders wenn viele Schritte benötigt werden.
- **env_size** bestimmt die Mindestzahl an Schritten bis zum Ziel (hier ist das Ziel `env_size - 1`). Größere Umgebungen führen bei gleicher Strategie im Mittel zu mehr Schritten und damit zu niedrigeren Returns.
- **p_random** erhöht zufällige Aktionen. Dadurch steigt meist die Zahl der Schritte und die Varianz der Returns, weil der Agent häufiger Umwege geht oder vom Ziel weg bewegt wird.
- **episodes** beeinflusst nur die Stabilität der Statistik: mehr Episoden ergeben eine glattere, zuverlässigere Verteilung, ändern aber nicht das Verhalten einer einzelnen Episode.
- **max_steps** setzt eine Obergrenze: Wenn das Ziel nicht rechtzeitig erreicht wird, endet die Episode trotzdem. Viele Abbrüche führen zu einer Häufung niedriger Returns.

In [39]:
import time
import random
import ipywidgets as w
from IPython.display import display

def render_grid(pos, goal, width, height, walls=None, trail=None):
    walls = walls or set()
    trail = trail or set()
    px, py = pos
    gx, gy = goal

    lines = []
    for y in range(height):
        row = []
        for x in range(width):
            cell = "."
            if (x, y) in walls:
                cell = "#"
            if (x, y) in trail:
                cell = "*"
            if (x, y) == (gx, gy):
                cell = "G"
            if (x, y) == (px, py):
                cell = "M"
            row.append(cell)
        lines.append(" ".join(row))
    return "\n".join(lines)

def manhattan(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def generate_world(width, height, density=0.18, seed=0):
    """
    Erzeugt ein Hindernisfeld mit garantierter Verbindung zwischen Start und Ziel:
    - Zuerst wird ein Pfad gebaut, dann werden zufällige Wände gesetzt (nicht auf dem Pfad).
    """
    rng = random.Random(seed)
    start = (0, 0)
    goal = (width-1, height-1)

    # Pfad grob: rechts/runter mit kleinen Zufallsvarianten
    path = {start}
    x, y = start
    while (x, y) != goal:
        moves = []
        if x < goal[0]: moves.append((x+1, y))
        if y < goal[1]: moves.append((x, y+1))
        # etwas Variation: manchmal zuerst runter statt rechts
        nxt = rng.choice(moves)
        x, y = nxt
        path.add((x, y))

    walls = set()
    for yy in range(height):
        for xx in range(width):
            if (xx, yy) in path:
                continue
            if (xx, yy) == start or (xx, yy) == goal:
                continue
            if rng.random() < density:
                walls.add((xx, yy))

    return start, goal, walls

def run_grid_visual(out, width=18, height=10, density=0.18, seed=0,
                    p_random=0.15, step_penalty=-0.02, max_steps=400, delay=0.04,
                    show_trail=True):
    # Welt generieren
    start, goal, walls = generate_world(width, height, density=density, seed=seed)

    def valid(p):
        x, y = p
        return 0 <= x < width and 0 <= y < height and p not in walls

    actions = [(1,0), (-1,0), (0,1), (0,-1)]

    def greedy_towards_goal(pos):
        # wähle eine Aktion, die Manhattan-Distanz reduziert (wenn möglich)
        best = []
        best_d = manhattan(pos, goal)
        for dx, dy in actions:
            nxt = (pos[0]+dx, pos[1]+dy)
            if not valid(nxt):
                continue
            d = manhattan(nxt, goal)
            if d < best_d:
                best = [(dx, dy)]
                best_d = d
            elif d == best_d:
                best.append((dx, dy))
        # fallback: irgendein valider Schritt
        if best:
            return random.choice(best)
        valid_actions = [(dx,dy) for dx,dy in actions if valid((pos[0]+dx, pos[1]+dy))]
        return random.choice(valid_actions) if valid_actions else (0,0)

    pos = start
    total = 0.0
    trail = set()

    with out:
        out.clear_output(wait=True)
        for t in range(max_steps):
            if show_trail:
                trail.add(pos)

            out.clear_output(wait=True)
            print(f"t={t:03d}  pos={pos}  goal={goal}  return={total:.2f}  walls={len(walls)}")
            print(render_grid(pos, goal, width, height, walls, trail if show_trail else None))

            # Aktion wählen
            if random.random() < p_random:
                dx, dy = random.choice(actions)
            else:
                dx, dy = greedy_towards_goal(pos)

            nxt = (pos[0]+dx, pos[1]+dy)
            if valid(nxt):
                pos = nxt

            # Reward
            if pos == goal:
                total += 1.0
                out.clear_output(wait=True)
                print(f"Ziel erreicht in {t+1} Schritten, return={total:.2f}")
                print(render_grid(pos, goal, width, height, walls, trail if show_trail else None))
                break
            else:
                total += step_penalty

            time.sleep(delay)

# UI
btn = w.Button(description="Run episode", button_style="primary")
W = w.IntSlider(value=20, min=10, max=40, step=1, description="width")
H = w.IntSlider(value=10, min=6, max=20, step=1, description="height")
D = w.FloatSlider(value=0.18, min=0.05, max=0.35, step=0.01, description="density")
S = w.IntSlider(value=0, min=0, max=50, step=1, description="seed")

P = w.FloatSlider(value=0.15, min=0.0, max=1.0, step=0.05, description="p_random")
SP = w.FloatSlider(value=-0.02, min=-0.2, max=0.0, step=0.01, description="penalty")
DL = w.FloatSlider(value=0.04, min=0.01, max=0.2, step=0.01, description="delay")
TR = w.Checkbox(value=True, description="trail")

out = w.Output()
display(w.VBox([
    w.HBox([btn]),
    w.HBox([W, H, D, S]),
    w.HBox([P, SP, DL, TR]),
    out
]))

def on_click(_):
    run_grid_visual(
        out,
        width=W.value, height=H.value, density=D.value, seed=S.value,
        p_random=P.value, step_penalty=SP.value, delay=DL.value,
        show_trail=TR.value
    )

btn.on_click(on_click)

### Interpretation der Skala und Parameterwirkung

- **Return** ist die aufsummierte Belohnung pro Episode: Bei Zielerreichung gibt es typischerweise **+1.0**, ansonsten wird pro Schritt eine **Schrittstrafe** addiert. Viele Schritte bedeuten deshalb meist einen deutlich kleineren Return.
- **step_penalty** (negativ) verschiebt die gesamte Episode nach unten: Je stärker die Schrittstrafe, desto stärker sinkt der Return bei Umwegen, Schleifen oder langsamer Zielerreichung.
- **width, height** erhöhen die Problemgröße: längere Wege und mehr mögliche Zustände. Ohne zusätzliche Intelligenz steigt die Chance auf Umwege und damit sinkt der Return im Mittel.
- **density** steuert die Hindernisdichte: mehr Wände bedeuten mehr Sackgassen und Umwege. Hohe Dichte erhöht typischerweise die Varianz der Returns, weil manche Läufe schnell eine gute Route finden und andere hängen bleiben.
- **seed** erzeugt ein anderes Layout: Damit lassen sich Szenarien reproduzieren oder gezielt variieren, ohne die Parameter zu verändern.
- **p_random** erhöht Zufallsschritte: Das kann helfen, lokale Blockaden zu lösen, führt aber bei zu hohem Wert zu ungerichtetem Verhalten, mehr Schritten und geringerem Return.
- **trail** zeigt die Spur des Agenten: Gut, um Umwege, Schleifen und „Festhängen“ sichtbar zu machen. Der Trail verändert nicht das Verhalten, nur die Visualisierung.
- **delay** beeinflusst nur die Darstellungsgeschwindigkeit, nicht das Entscheidungsverhalten.

In [34]:
# Mini-Gridworld (Agenten-Notebook -> später 1:1 für Q-Learning im RL-Notebook)
# Ziel: gleiche API wie in RL: reset() -> state, step(action) -> next_state, reward, done, info

import random
import numpy as np

class MiniGridworld:
    """
    Zustände: (x, y) als Tupel
    Aktionen: 0=up, 1=right, 2=down, 3=left
    Rewards: step_penalty pro Schritt, goal_reward beim Ziel
    Hindernisse: walls als Set[(x,y)]
    """
    ACTIONS = {
        0: (0, -1),  # up
        1: (1, 0),   # right
        2: (0, 1),   # down
        3: (-1, 0),  # left
    }
    ACTION_NAMES = {0:"up", 1:"right", 2:"down", 3:"left"}

    def __init__(self, width=8, height=6, start=(0,0), goal=(7,5),
                 step_penalty=-0.02, goal_reward=1.0, max_steps=200,
                 walls=None, seed=0):
        self.width = width
        self.height = height
        self.start = start
        self.goal = goal
        self.step_penalty = step_penalty
        self.goal_reward = goal_reward
        self.max_steps = max_steps
        self.walls = set(walls) if walls else set()
        self.rng = random.Random(seed)

        # Sicherheitschecks
        assert self._in_bounds(self.start) and self.start not in self.walls
        assert self._in_bounds(self.goal) and self.goal not in self.walls

        self.reset()

    def _in_bounds(self, p):
        x, y = p
        return 0 <= x < self.width and 0 <= y < self.height

    def reset(self):
        self.pos = self.start
        self.t = 0
        return self.pos

    def step(self, action):
        self.t += 1
        dx, dy = self.ACTIONS[action]
        nxt = (self.pos[0] + dx, self.pos[1] + dy)

        # Wenn ungültig oder Wand: bleiben
        if (not self._in_bounds(nxt)) or (nxt in self.walls):
            nxt = self.pos

        self.pos = nxt

        done = False
        reward = self.step_penalty

        if self.pos == self.goal:
            reward = self.goal_reward
            done = True

        if self.t >= self.max_steps:
            done = True

        info = {"t": self.t}
        return self.pos, reward, done, info

    def render(self, trail=None):
        trail = trail or set()
        lines = []
        for y in range(self.height):
            row = []
            for x in range(self.width):
                p = (x, y)
                c = "."
                if p in self.walls: c = "#"
                if p in trail: c = "*"
                if p == self.goal: c = "G"
                if p == self.pos: c = "M"
                row.append(c)
            lines.append(" ".join(row))
        return "\n".join(lines)

    def state_id(self, state=None):
        """
        Für Q-Learning praktisch: mappe (x,y) -> int [0..width*height-1]
        """
        if state is None:
            state = self.pos
        x, y = state
        return y * self.width + x

    def id_to_state(self, sid):
        x = sid % self.width
        y = sid // self.width
        return (x, y)


# 1) Beispielwelt, die später exakt so im RL-Notebook wiederverwendet werden kann
walls = {
    (2,1),(2,2),(2,3),
    (4,2),(5,2),
    (6,0),(6,1)
}

env = MiniGridworld(
    width=10, height=6,
    start=(0,0), goal=(9,5),
    step_penalty=-0.02, goal_reward=1.0,
    max_steps=120,
    walls=walls,
    seed=0
)

print(env.render())

# 2) Kurzer "Agentenlauf" (greedy + etwas Zufall), nur zur Visualisierung
def greedy_action(state, goal):
    # wählt eine Aktion, die Manhattan-Distanz reduziert, sonst zufällig
    x, y = state
    gx, gy = goal
    candidates = []
    if gy < y: candidates.append(0)  # up
    if gx > x: candidates.append(1)  # right
    if gy > y: candidates.append(2)  # down
    if gx < x: candidates.append(3)  # left
    return random.choice(candidates) if candidates else random.choice([0,1,2,3])

p_random = 0.2
delay = 0.08  # nur relevant, wenn du später sleep + clear_output nutzt
trail = set()

state = env.reset()
total = 0.0
done = False

# Ausgabe ohne Animation (nur "Trace" als Text)
for _ in range(40):
    trail.add(state)
    if random.random() < p_random:
        action = random.choice([0,1,2,3])
    else:
        action = greedy_action(state, env.goal)

    state, r, done, info = env.step(action)
    total += r
    if done:
        break

print("\nReturn:", round(total, 3), "Steps:", info["t"])
print(env.render(trail=trail))

M . . . . . # . . .
. . # . . . # . . .
. . # . # # . . . .
. . # . . . . . . .
. . . . . . . . . .
. . . . . . . . . G

Return: 0.28 Steps: 37
* . . . . . # . . .
* * # . . . # . . .
. * # . # # . . . .
. * # . . . . . . .
. * . . . . . . . .
. * * * * * * * * M


In [36]:
import time
import random
import ipywidgets as w
from IPython.display import display

# --- Hilfen: zufällige Wände mit garantiert freiem Start/Ziel ---
def generate_random_walls(width, height, start, goal, density=0.18, seed=0):
    rng = random.Random(seed)
    walls=set()
    for y in range(height):
        for x in range(width):
            p=(x,y)
            if p == start or p == goal:
                continue
            if rng.random() < density:
                walls.add(p)
    return walls

def choose_action_random():
    return random.choice([0,1,2,3])

def choose_action_greedy(state, goal):
    x, y = state
    gx, gy = goal
    preferred = []
    if gy < y: preferred.append(0)  # up
    if gx > x: preferred.append(1)  # right
    if gy > y: preferred.append(2)  # down
    if gx < x: preferred.append(3)  # left
    return random.choice(preferred) if preferred else random.choice([0,1,2,3])

def run_episode(out,
                width=12, height=8, density=0.18, seed=0,
                step_penalty=-0.02, goal_reward=1.0,
                policy="greedy", p_random=0.2,
                max_steps=200, delay=0.06, show_trail=True):

    start=(0,0)
    goal=(width-1, height-1)
    walls = generate_random_walls(width, height, start, goal, density=density, seed=seed)

    # Env neu erzeugen (wichtig: 1:1 gleiche API später für Q-Learning)
    env = MiniGridworld(
        width=width, height=height,
        start=start, goal=goal,
        step_penalty=step_penalty, goal_reward=goal_reward,
        max_steps=max_steps,
        walls=walls,
        seed=seed
    )

    state = env.reset()
    total = 0.0
    trail = set()

    with out:
        out.clear_output(wait=True)

        for t in range(max_steps):
            if show_trail:
                trail.add(state)

            out.clear_output(wait=True)
            print(f"t={t:03d}  state={state}  return={total:.2f}  walls={len(walls)}")
            print(env.render(trail=trail if show_trail else None))

            # Aktion wählen
            if policy == "random":
                action = choose_action_random()
            else:
                if random.random() < p_random:
                    action = choose_action_random()
                else:
                    action = choose_action_greedy(state, env.goal)

            state, r, done, info = env.step(action)
            total += r

            time.sleep(delay)
            if done:
                out.clear_output(wait=True)
                print(f"Ende: steps={info['t']}  return={total:.2f}  reached_goal={state==env.goal}  walls={len(walls)}")
                print(env.render(trail=trail if show_trail else None))
                break

# --- UI ---
btn = w.Button(description="Run episode", button_style="primary")

W = w.IntSlider(value=20, min=10, max=40, step=1, description="width")
H = w.IntSlider(value=10, min=5, max=20, step=1, description="height")
D = w.FloatSlider(value=0.18, min=0.00, max=0.40, step=0.02, description="density")
S = w.IntSlider(value=0, min=0, max=99, step=1, description="seed")

SP = w.FloatSlider(value=-0.02, min=-0.30, max=0.00, step=0.01, description="penalty")
GR = w.FloatSlider(value=1.00, min=0.20, max=2.00, step=0.10, description="goal_reward")

policy = w.Dropdown(options=[("Greedy (mit Zufall)", "greedy"), ("Nur Zufall", "random")],
                    value="greedy", description="policy")
P = w.FloatSlider(value=0.20, min=0.0, max=1.0, step=0.05, description="p_random")

MS = w.IntSlider(value=200, min=30, max=600, step=10, description="max_steps")
DL = w.FloatSlider(value=0.06, min=0.01, max=0.30, step=0.01, description="delay")
TR = w.Checkbox(value=True, description="trail")

out = w.Output()

display(w.VBox([
    w.HBox([btn]),
    w.HBox([W, H, D, S]),
    w.HBox([SP, GR]),
    w.HBox([policy, P]),
    w.HBox([MS, DL, TR]),
    out
]))

def on_click(_):
    run_episode(
        out,
        width=W.value, height=H.value, density=D.value, seed=S.value,
        step_penalty=SP.value, goal_reward=GR.value,
        policy=policy.value, p_random=P.value,
        max_steps=MS.value, delay=DL.value, show_trail=TR.value
    )

btn.on_click(on_click)

### MiniGridworld: Bedeutung der Parameter

- **width, height** bestimmen die Größe der Welt und damit die Mindestlänge des Weges zum Ziel. Größere Welten führen meist zu mehr Schritten und damit zu einem niedrigeren Return.
- **density** steuert die Hindernisdichte. Höhere Dichte erzeugt mehr Umwege und Sackgassen, erhöht die Varianz der Läufe und senkt typischerweise den Return.
- **seed** erzeugt ein reproduzierbares Layout. Gleiche Parameter mit gleichem Seed liefern dieselbe Welt, unterschiedliche Seeds liefern neue Szenarien.
- **step_penalty** ist die Schrittstrafe. Je stärker negativ, desto „teurer“ sind Umwege und desto wichtiger wird ein kurzer Weg.
- **goal_reward** ist die Belohnung beim Erreichen des Ziels. Höhere Werte heben die Returns insgesamt an, ändern aber nicht, dass Umwege durch die Schrittstrafe bestraft werden.
- **policy und p_random** steuern das Verhalten. Greedy versucht in Richtung Ziel zu gehen, `p_random` fügt Zufallsschritte hinzu, was aus Sackgassen helfen kann, aber bei zu hohem Wert zu unnötigen Umwegen führt.
- **max_steps** begrenzt die Episodenlänge. Wenn das Ziel nicht erreicht wird, endet die Episode dennoch, meist mit niedrigem Return.